**For comparing two locations**

Based on Folium docs at https://python-visualization.github.io/folium/latest/index.html

In [1]:
import time
import folium
import pandas as pd
from IPython.display import clear_output

#file = input("Enter the excel file name: ")
file = "20rows.xlsx"
#data are specimens sorted with two locations > 50km apart. Maybe reduce this to some other distance?
df = pd.read_excel(file)

#output dataframe to be same as input but with two extra columns to take chosen lat/long
dfout = df

#add columns for choice of lat/long
dfout['SelectLat'] = ''
dfout['SelectLong'] = ''

#print(f"No. specimens is {nrows}")

In [2]:
#needed since the 'spec' variable doesn't seem to remain an integer all the time. Not sure why. 
def is_int(input_str):
    try:
        int(input_str)
        return True
    except ValueError:
        return False

In [3]:
#set specimen number based on entered value
def set_row(inp, row, nrows):
    #print(f"row in set_spec = {row}")
    if is_int(inp):
        #print(f"'{inp}' is an integer.")
        row = inp
    else:
        #increment if desired
        if inp == "n": 
            row += 1
        elif inp == "p":
            row -= 1
        elif inp == "f":
            row = 1
        elif inp == "l":
            row = nrows
        #if g, o, d, or u - don't change row
        else:
            row = row
    return int(row)

In [4]:
def make_map(row, df):
    #print(f"row is {row}")
    lat1 = df.OrigLat[row-1]
    lon1 = df.OrigLong[row-1]
    lat2 = df.ApLat[row-1]
    lon2 = df.ApLong[row-1]
    trueSpec = df.Specimen[row-1]
    #distance = df.distance[row-1]
    
    if lat1 == 0 and lat2 == 0:
        m = folium.Map(location=[lat2, lon2], zoom_start=5, control_scale=True)
        folium.Marker(
            location=[0, 0],             
            tooltip = "No Location found", 
            icon=folium.Icon(color="red"),
        ).add_to(m)  
    
    elif lat1 == 0 :
        m = folium.Map(location=[lat2, lon2], zoom_start=5, control_scale=True)
        folium.Marker(
            location=[lat2, lon2], 
            tooltip = f"Specimen ID {trueSpec}; no original loc'n.",
            popup=f"GeoApp: {lat2}, {lon2}", 
            icon=folium.Icon(color="green"),
        ).add_to(m)        

    elif lat2 == 0 :
        m = folium.Map(location=[lat1, lon1], zoom_start=5, control_scale=True)
        folium.Marker(
            location=[lat1, lon1], 
            tooltip = f"Specimen ID {trueSpec}; no GeoApp loc'n.",
            popup=f"Original: {lat1}, {lon1}",
            icon=folium.Icon(color="blue"),
        ).add_to(m)
    
    else:        
        m = folium.Map(location=[lat2, lon2], zoom_start=5,control_scale=True)
        #see https://python-visualization.github.io/folium/latest/user_guide/features/fit_overlays.html
        fg1 = folium.FeatureGroup().add_to(m)
        folium.Marker(
            location=[lat1, lon1], 
            tooltip = f"Specimen ID {trueSpec}",
            popup=f"Original: {lat1}, {lon1}",
            icon=folium.Icon(color='blue', icon='1', prefix='fa'),
        ).add_to(fg1)
        
        fg2 = folium.FeatureGroup().add_to(m)
        folium.Marker(
            location=[lat2, lon2], 
            tooltip = f"Specimen ID {trueSpec}",
            popup=f"GeoApp: {lat2}, {lon2}", 
            icon=folium.Icon(color='green', icon='2', prefix='fa'),
        ).add_to(fg2)  

        folium.FitOverlays(max_zoom=7, padding=50).add_to(m)
        folium.LayerControl().add_to(m)
        
    return m

In [5]:
def is_int(n):
    try:
        int(n)
    except ValueError:
        return False
    else:
        return True

In [6]:
#Print the action taken based on input choice
def set_choice(inp):
    if inp == "2":
        print(f"GEOAPPIFY'S LOCATION (green 2) is kept.\n")
        dfout.loc[row-1, 'SelectLat'] = df.loc[row-1, 'ApLat']
        dfout.loc[row-1, 'SelectLong'] = df.loc[row-1, 'ApLong']
    elif inp == "1":
        print(f"ORIGINAL LOCATION (blue 1) is kept.\n")
        dfout.loc[row-1, 'SelectLat'] = df.loc[row-1, 'OrigLat']
        dfout.loc[row-1, 'SelectLong'] = df.loc[row-1, 'OrigLong']
    elif inp == "d":
        print(f"LOCATION flagged is to be decided later.\n")
        dfout.loc[row-1, 'SelectLat'] = "undecided"
        dfout.loc[row-1, 'SelectLong'] = "undecided"
    elif inp == "u":
        print(f"LOCATION is set to unknown (lat/lon = 0/0).\n")
        dfout.loc[row-1, 'SelectLat'] = 0.0
        dfout.loc[row-1, 'SelectLong'] = 0.0
    return
    

In [7]:
row = 1
row_OK = True
nrows = df.shape[0]
inp = "c"

while inp != "x":
    clear_output(wait=True)
#Plot the map
    m = make_map(row,df)
    display(m)
    #m.save("map.html")

#Print current specimen's details below the map.
    separation = df.distance[row-1]
    if is_int(separation): #may be char or string if there are not 2 locations.
        print(f"Specimen ID {df.Specimen[row-1]} in row {row}; loc'ns are ~{int(separation)}km apart.") 
    else:
        print(f"Specimen ID {df.Specimen[row-1]} in row {row}. One loc'n only.")
    print(f"ADDR={df.address[row-1]}, CITY={df.City[row-1]}, STATE={df.State[row-1]}, COUNTRY={df.Country[row-1]}.\n")
    time.sleep(.5)

#Update loc'n choice, continue or exit
    inp = ""
    while inp not in ["1", "2", "d", "u", "c", "x",]:
        inp = input(f"Keep ... (1)blue = original, (2)green = GeoApify, (d)ecide later, (u)nknown (set to 0), (c)ontinue:")            
    
    if inp in ["1", "2", "d", "u"]:
        set_choice(inp)
    elif inp == "x":  #Quite if requested
        print("all done")
        break
    
    #default after the while loop and if statement will be inp = "c"
    
#Fetch next specimen choice
    row_OK = False
    row_temp = row
    while not row_OK:
        inp = input(f"ENTER row # (1 to {nrows}) or (n)ext, (p)rev, (f)irst, (l)ast;\n")

#set row value based on input value (row #) or character
        row = set_row(inp, int(row), nrows)
        if row < 1 or row > nrows:
            print(f">Row choice must be between 1 and {nrows}. Please try again.")
            row_OK = False
            row = row_temp
        else:
            row_OK = True
    
#continue in while loop if not exit
    

Specimen ID 3113 in row 19; loc'ns are ~7107km apart.
ADDR=De L' Etoile, CITY=nan, STATE=Katanga Province, COUNTRY=Dem. Rep. Of Congo.

Keep ... (1)blue = original, (2)green = GeoApify, (d)ecide later, (u)nknown (set to 0), (c)ontinue:1
ORIGINAL LOCATION (blue 1) is kept.

ENTER row # (1 to 20) or (n)ext, (p)rev, (f)irst, (l)ast;
x


In [8]:
#save results 
#outf = open("ChosenGeolocns.xlsx", "w", encoding="utf-16") #re-write, NOT appending.
dfout.to_excel("ChosenGeolocns.xlsx")
#outf.close()

clear_output(wait=True)
print("Choices saved to file ChosenGeolocns.xlsx.")

All done.
